In [1]:
include("../src/Conduction.jl");
using Plots, LaTeXStrings;

In [2]:
semiconductor = Conduction.Semiconductor(
1.38 * 10^-23, # Boltzman constant (J.K^-1)
1.6*10^-19, # Electron's charge (C)
10^7, # decay constant of the assumed hydrogen-like localized state wave functions (cm^-1)
0.1 * 1.6*10^-19, # Mode effect of the phonons (J)
2.1 * 10^18, # intrinsic semiconductor's density (cm^-3)
0, # Doping states' density (cm^-3)
0.1 * 1.6 * 10^-19, # Energy to a vacant target site (J)
-2*10^3, # Field (V.cm^-1)
10^13, # Base electron jump rate
-10.0, # Fermi level (J)
2.3, # Intrinsic semiconductor's gaussian width (J)
2.7, # Doping states' gaussian width (J)
2.7, # Amount of disorder
1 # Phonon lower frequency
);

T = 300;
range = -15:1:15;

In [3]:
semiconductor.Uf(T) / semiconductor.SigmaI(T)

-4.3478260869565215

Compared to papers, the ratio seems too high

In [4]:
Conduction.overallEin(semiconductor, Conduction.RnnVRH, T, 15) * semiconductor.k * T / semiconductor.q

3.2951394948128266e6

In [6]:
println(semiconductor.Uf(T) / (1.6 * 10^-19))
println(semiconductor.SigmaI(T) / (1.6 * 10^-19))

-0.25875
0.059512499999999996


Fermi level is too high.

In [10]:
semiconductor.Uf(T) / semiconductor.SigmaI(T)

-3.7037037037037033

 We'll find $E_F$ in such way that $\dfrac{E_F}{\sigma} = -6.2$

In [8]:
6.7 * semiconductor.SigmaI(T) / semiconductor.k / T

14.26

In [12]:
semiconductor.Uf(T) = -14.26 * semiconductor.k * T;
Conduction.overallEin(semiconductor, Conduction.RnnVRH, T, 15) * semiconductor.k * T / semiconductor.q

3.31060033468195e6

We'll take $E_F$ low compared to the otger values.

In [13]:
2 * semiconductor.SigmaI(T) / semiconductor.k / T

4.6

In [14]:
semiconductor.Uf(T) = -4.6 * semiconductor.k * T;
Conduction.overallEin(semiconductor, Conduction.RnnVRH, T, 15) * semiconductor.k * T / semiconductor.q

3.1479635703988867e6